In [22]:
from MOT import MOT
import numpy as np
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
import open3d as op3
# from open3d.web_visualizer import draw
from Utils import *
import pandas as pd
from scipy.sparse import csr_matrix
import json
from scipy.optimize import linear_sum_assignment
import pandas as pd
import matplotlib.pyplot as plt

In [64]:
RawTrajs = []
for i in range(4):
    RawTrajs_temp_0 = pd.read_csv('../RawLidarData/McCarranEvans_Train_{}/OutputFile/OutputTrajs/Trajctories_0.csv'.format(i))
    RawTrajs_temp_1 = pd.read_csv('../RawLidarData/McCarranEvans_Train_{}/OutputFile/OutputTrajs/Trajctories_1.csv'.format(i))
    RawTrajs.append(RawTrajs_temp_0)
    RawTrajs.append(RawTrajs_temp_1)

In [65]:
TotalTrajs = []
for rawtraj in RawTrajs:
    TrajGroups = rawtraj.groupby('ObjectID')
    for g in TrajGroups:
        TotalTrajs.append(g[1])

In [72]:
for time_step in [1,2,3,4,5,6,7,8,9,10]:
#     time_step = 1
    Speed_curs = []
    Coord_curs = []
    Coord_nexts = []
    for ind in tqdm(range(len(TotalTrajs))):
        temp_group = TotalTrajs[ind]
        Coords = np.array(temp_group.loc[:,['Coord_X_Mea','Coord_Y_Mea']])
        if len(Coords) < (time_step + 2):
                continue
        for i in range(time_step,len(Coords) - time_step):
            Coord_cur = Coords[i]
            Coord_next = Coords[i + time_step]
            
            if np.isnan(Coord_cur).any()|np.isnan(Coord_next).any():
                continue
                
            if np.sqrt(np.sum(Coord_next - Coord_cur)**2) > 4.1 * time_step:
                continue
                
            Speed_cur = Coords[i] - Coords[i - time_step]
            
            if np.sqrt(np.sum((Speed_cur**2))) > 4.1 * time_step:
                continue 
            
            Speed_curs.append(Speed_cur)
            Coord_curs.append(Coord_cur)
            Coord_nexts.append(Coord_next)
    col_names = ['X_cur','Y_cur', 'X_speed_cur','Y_speed_cur','X_next','Y_next']
    result = pd.DataFrame(np.concatenate([np.array(Coord_curs),np.array(Speed_curs),np.array(Coord_nexts)],axis = 1),columns=col_names)
    result.to_csv('../RawLidarData/PosterierTrainData/Train_{}.csv'.format(time_step),index = False)
    print(len(result))

100%|██████████████████████████████████████| 5476/5476 [00:12<00:00, 449.11it/s]


561213


100%|██████████████████████████████████████| 5476/5476 [00:11<00:00, 459.25it/s]


554270


100%|██████████████████████████████████████| 5476/5476 [00:11<00:00, 464.62it/s]


...

502184


100%|██████████████████████████████████████| 5476/5476 [00:12<00:00, 455.75it/s]


494284


100%|██████████████████████████████████████| 5476/5476 [00:12<00:00, 450.07it/s]


486202
